In [4]:
from mjol import gan as mgan
from collections import defaultdict
import os

In [5]:
fn = "/ccb/salz4-3/hji20/hg002-q100-annotation/results/6_add_copies/mat_loff/fmted.modified.gff"
gan = mgan.GAn(
    file_name = fn,
    file_fmt = 'gff'
)
gan.build_db()

In [6]:
def is_coding_gene(f):
    for tx in f.children:
        for c in tx.children:
            if c.feature_type == "CDS":
                return True
    return False

In [7]:
copy_numbers_by_gname = defaultdict(int)
gene_chrs = {}
for uid, f in gan.features.items():
    if f.feature_type == "gene":
        if is_coding_gene(f):
            if 'gene_name' in f.attributes:
                gname = f.attributes['gene_name']
                copy_numbers_by_gname[gname] += 1
                if gname in gene_chrs:
                    gene_chrs[gname].append(f.chr)
                else:
                    gene_chrs[gname] = [f.chr]
            else:
                print(f)

In [5]:
fn = "/ccb/salz4-3/hji20/hg002-q100-annotation/results/6_add_copies/pat_loff/fmted.modified.gff"

gan2 = mgan.GAn(
    file_name = fn,
    file_fmt = 'gff'
)
gan2.build_db()

In [9]:
copy_numbers_by_gname2 = defaultdict(int)
gene_chrs2 = {}
for uid, f in gan2.features.items():
    if f.feature_type == "gene":
        if is_coding_gene(f):
            if 'gene_name' in f.attributes:
                gname = f.attributes['gene_name']
                copy_numbers_by_gname2[gname] += 1
                if gname in gene_chrs2:
                    gene_chrs2[gname].append(f.chr)
                else:
                    gene_chrs2[gname] = [f.chr]
            else:
                print(f)

In [17]:
union_gnames = set(copy_numbers_by_gname.keys()).union(set(copy_numbers_by_gname2.keys()))

out_fn = "/ccb/salz4-3/hji20/hg002-q100-annotation/results/analysis/copy_numbers.tsv"
with open(out_fn, 'w') as fh:
    fh.write(f'gene_name\tnum_m\tchroms_m\tpat\tnum_p\n')
    for gname in union_gnames:
        s = f'{gname}\t'
        if gname not in copy_numbers_by_gname:
            s += '0\tNone\t'
        else:
            s += f'{copy_numbers_by_gname[gname]}\t{','.join(gene_chrs[gname])}\t'
        
        if gname not in copy_numbers_by_gname2:
            s += '0\tNone'
        else:
            s += f'{copy_numbers_by_gname2[gname]}\t{','.join(gene_chrs2[gname])}'
        fh.write(s + '\n')